# 02 - Kaggle Training: Place Recognition on NCLT

Train a MinkLoc3D-based place recognition model on the NCLT dataset.

**Requirements**: Attach the [NCLT dataset](https://www.kaggle.com/datasets/creatorofuniverses/nclt-iprofi-hack-23) to this notebook.

## 1. Setup

In [ ]:
# Clone the project repo (adjust URL)
# !git clone https://github.com/YOUR_USERNAME/nclt-slam-project.git
# %cd nclt-slam-project

# Install dependencies
!pip install -q torch torchvision open3d pyyaml tqdm scipy scikit-learn matplotlib
# MinkowskiEngine requires special installation:
# !pip install -q MinkowskiEngine -f https://nvidia.com/MinkowskiEngine/cu118/torch2.0.0/index.html

import sys
sys.path.insert(0, '.')

## 2. Link Kaggle Data

In [ ]:
import os
from pathlib import Path

KAGGLE_DATA = Path('/kaggle/input/nclt-iprofi-hack-23/NCLT_preprocessed')
LOCAL_DATA = Path('./data/NCLT_preprocessed')

if KAGGLE_DATA.exists():
    print(f'Running on Kaggle. Data at: {KAGGLE_DATA}')
    # Create symlink for unified access
    LOCAL_DATA.parent.mkdir(parents=True, exist_ok=True)
    if not LOCAL_DATA.exists():
        os.symlink(str(KAGGLE_DATA), str(LOCAL_DATA))
    data_path = KAGGLE_DATA
elif LOCAL_DATA.exists():
    print(f'Running locally. Data at: {LOCAL_DATA}')
    data_path = LOCAL_DATA
else:
    raise FileNotFoundError(
        'Dataset not found. On Kaggle, attach the NCLT dataset. '
        'Locally, run: python scripts/download_nclt_sample.py'
    )

# List available sessions
sessions_dir = data_path / 'sessions'
if sessions_dir.exists():
    sessions = sorted([d.name for d in sessions_dir.iterdir() if d.is_dir()])
    print(f'Available sessions ({len(sessions)}): {sessions}')

## 3. Verify Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from src.utils.point_cloud import load_velodyne_bin
from src.utils.io_utils import load_config

config = load_config('configs/dataset_config.yaml')

# Load a sample point cloud
sample_session = sessions[0] if sessions else '2012-01-08'
velodyne_dir = data_path / 'sessions' / sample_session / 'velodyne'

if velodyne_dir.exists():
    bin_files = sorted(velodyne_dir.glob('*.bin'))
    if bin_files:
        sample_pc = load_velodyne_bin(bin_files[0])
        print(f'Sample point cloud: {sample_pc.shape}')
        print(f'X range: [{sample_pc[:,0].min():.1f}, {sample_pc[:,0].max():.1f}]')
        print(f'Y range: [{sample_pc[:,1].min():.1f}, {sample_pc[:,1].max():.1f}]')
        print(f'Z range: [{sample_pc[:,2].min():.1f}, {sample_pc[:,2].max():.1f}]')
        
        # Visualize
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        ax.scatter(sample_pc[:, 0], sample_pc[:, 1], s=0.1, c=sample_pc[:, 2], cmap='viridis')
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_title(f'Sample Point Cloud - {sample_session}')
        ax.set_aspect('equal')
        plt.show()
    else:
        print('No .bin files found')
else:
    print(f'Velodyne directory not found: {velodyne_dir}')

## 4. Load Training Config

In [ ]:
train_config = load_config('configs/train_config.yaml')
training = train_config['training']

print('Training configuration:')
for key, value in training.items():
    print(f'  {key}: {value}')

## 5. Training Loop

Train the place recognition model with triplet loss.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

# Choose model based on MinkowskiEngine availability
try:
    import MinkowskiEngine as ME
    model_type = 'minkloc3d'
    print('MinkowskiEngine available - using MinkLoc3D')
except ImportError:
    model_type = 'pointnet'
    print('MinkowskiEngine not available - falling back to PointNet')

from src.models.place_recognition import PlaceRecognitionWrapper, TripletLoss

feature_dim = training.get('feature_dim', 256)
model = PlaceRecognitionWrapper(
    model_type=model_type,
    feature_dim=feature_dim,
).to(device)

loss_fn = TripletLoss(margin=training.get('margin', 0.2))
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=training.get('learning_rate', 1e-3),
    weight_decay=training.get('weight_decay', 1e-4),
)

print(f'Model: {model_type}, feature_dim={feature_dim}')
total_params = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total_params:,}')

In [ ]:
# TODO: Implement actual training loop
# This is a placeholder - fill in after data loading is verified

# from src.datasets.nclt_pairs import NCLTPairsDataset, pairs_collate_fn
# from src.datasets.transforms import build_transforms
#
# aug_config = {
#     'point_cloud': config['nclt']['point_cloud'],
#     'augmentation': training.get('augmentation', {}),
# }
# train_transform = build_transforms(aug_config, is_train=True)
#
# train_dataset = NCLTPairsDataset(
#     config_path='configs/dataset_config.yaml',
#     split='train',
#     transform=train_transform,
# )
#
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=training.get('batch_size', 32),
#     shuffle=True,
#     num_workers=2,
#     collate_fn=pairs_collate_fn,
# )
#
# epochs = training.get('epochs', 80)
# for epoch in range(epochs):
#     model.train()
#     epoch_loss = 0.0
#     for batch in train_loader:
#         anchor_desc = model(batch['anchor'].to(device))
#         positive_desc = model(batch['positive'].to(device))
#         negative_desc = model(batch['negatives'].to(device))
#         loss = loss_fn(anchor_desc, positive_desc, negative_desc)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         epoch_loss += loss.item()
#     print(f'Epoch {epoch}: loss={epoch_loss/len(train_loader):.4f}')

print('Training loop placeholder - uncomment when data is ready')

## 6. Evaluation

In [ ]:
# from src.evaluation.metrics import recall_at_k
# from src.evaluation.visualization import plot_recall_at_k
#
# # Extract descriptors for validation set
# model.eval()
# descriptors = []
# positions = []
#
# with torch.no_grad():
#     for batch in val_loader:
#         desc = model(batch['anchor'].to(device))
#         descriptors.append(desc.cpu().numpy())
#         positions.append(batch['anchor_pose'][:, :3, 3].numpy())
#
# descriptors = np.concatenate(descriptors)
# positions = np.concatenate(positions)
#
# # Compute recall@K
# n = len(descriptors)
# mid = n // 2
# recall = recall_at_k(
#     descriptors[:mid], descriptors[mid:],
#     positions[:mid], positions[mid:],
#     k_values=[1, 5, 10],
# )
# print(f'Recall: {recall}')
# plot_recall_at_k(recall)
# plt.show()

print('Evaluation placeholder - uncomment after training')

## 7. Save Checkpoints

In [ ]:
# Save to Kaggle output
import os

output_dir = Path('/kaggle/working') if Path('/kaggle/working').exists() else Path('./checkpoints')
output_dir.mkdir(parents=True, exist_ok=True)

# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'config': training,
# }, output_dir / 'best_model.pth')
# print(f'Model saved to {output_dir / "best_model.pth"}')

print(f'Checkpoint directory: {output_dir}')
print('Uncomment save code after training is implemented')